In [15]:
import requests
from requests import ConnectionError,HTTPError,Timeout
import numpy as np
import pandas as pd

url = 'https://data.ntpc.gov.tw/api/datasets/010e5b15-3823-4b20-b401-b1cf000550c5/json?page=0&size=1000'
try:
    response = requests.get(url)    
    response.raise_for_status()    
except ConnectionError:
    print('找不到伺服器')
except HTTPError:
    print('網頁找不到')
except Timeout:
    print('超過時間沒有回應')
else:
    print('沒有發生問題')

records = response.json()
#records
allRecords = pd.DataFrame(records,columns=['sna','tot','mday','sbi','sarea','ar','bemp'])
allRecords1 = allRecords.rename(columns= {'sna':'站名', 'sarea':'區域', 'ar':'地址', 'tot':'數量','sbi':'可借', 'bemp':'可還','mday':'時間'})
#allRecords1

沒有發生問題


In [16]:
allRecords1['站名'].apply(lambda name:name[11:])

0             下庄市場
1           八里行政中心
2       八里中庄市場綜合大樓
3             大崁國小
4            龍形停車場
          ...     
995          深坑松柏街
996    深坑區公所(深坑老街)
997         東南科技大學
998          深坑雲鄉路
999        埔新街61巷口
Name: 站名, Length: 1000, dtype: object

In [17]:
allRecords1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   站名      1000 non-null   object
 1   數量      1000 non-null   object
 2   時間      1000 non-null   object
 3   可借      1000 non-null   object
 4   區域      1000 non-null   object
 5   地址      1000 non-null   object
 6   可還      1000 non-null   object
dtypes: object(7)
memory usage: 54.8+ KB


In [18]:
allRecords1['時間'] = pd.to_datetime(allRecords1['時間'])

In [19]:
allRecords1[['數量','可借','可還']] = allRecords1[['數量','可借','可還']].astype(int)

In [20]:
allRecords1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   站名      1000 non-null   object        
 1   數量      1000 non-null   int64         
 2   時間      1000 non-null   datetime64[ns]
 3   可借      1000 non-null   int64         
 4   區域      1000 non-null   object        
 5   地址      1000 non-null   object        
 6   可還      1000 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 54.8+ KB


In [29]:
import numpy as np
grouped = allRecords1.groupby('區域')
#grouped[['數量','可借','可還']].sum()
agg_df = grouped[['數量','可借','可還']].agg([("加總","sum"),("平均","mean"),("中間數","median"),("最多","max"),("最少","min")])

In [35]:
agg_df.columns.names = ["租借","統計"]#讓數量及加總前有名稱
s1 = agg_df.stack(level=['租借','統計'],future_stack=True)
s1

區域   租借  統計 
三峽區  數量  加總     1540.000000
         平均       22.318841
         中間數      20.000000
         最多       63.000000
         最少       10.000000
                   ...     
金山區  可還  加總      125.000000
         平均       17.857143
         中間數      15.000000
         最多       28.000000
         最少        9.000000
Length: 315, dtype: float64

In [37]:
s1.unstack(level='統計')

統計          加總         平均   中間數    最多    最少
區域  租借                                     
三峽區 數量  1540.0  22.318841  20.0  63.0  10.0
    可借   675.0   9.782609   8.0  30.0   0.0
    可還   865.0  12.536232  10.0  42.0   0.0
三芝區 數量   170.0  21.250000  16.0  48.0  14.0
    可借    44.0   5.500000   5.5  11.0   2.0
...        ...        ...   ...   ...   ...
貢寮區 可借    20.0   6.666667   3.0  16.0   1.0
    可還    40.0  13.333333  17.0  19.0   4.0
金山區 數量   163.0  23.285714  24.0  30.0  15.0
    可借    38.0   5.428571   6.0   9.0   2.0
    可還   125.0  17.857143  15.0  28.0   9.0

[63 rows x 5 columns]